In [1]:
!nvidia-smi

Thu Oct 14 15:14:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install lightning-flash
!pip install 'lightning-flash[image]'

In [3]:
import pandas as pd
import torch
import flash
from flash.image import ImageClassificationData, ImageClassifier

In [4]:
# Create Data Module
PATH = "/content/drive/MyDrive/gh/kaggle/dacon/img_clf"
# "/train_imgs"
# "/test_imgs"
# "/sample_submission.csv"
# "/test.csv"
# "/train.csv"

In [5]:
train_df = pd.read_csv(PATH + "/train.csv")
test_df = pd.read_csv(PATH + "/test.csv")

In [6]:
test_df

,uid,img_path
0,20000,test_imgs/20000.jpg
1,20001,test_imgs/20001.jpg
2,20002,test_imgs/20002.jpg
3,20003,test_imgs/20003.jpg
4,20004,test_imgs/20004.jpg
...,...,...
4745,24745,test_imgs/24745.jpg
4746,24746,test_imgs/24746.jpg
4747,24747,test_imgs/24747.jpg
4748,24748,test_imgs/24748.jpg


In [7]:
train_df.img_path[1]

'train_imgs/10001.jpg'

In [8]:
train_files = []
predict_files = []

for path in train_df.img_path:
    train_files.append(PATH + f"/{path}")

for path in test_df.img_path:
    predict_files.append(PATH + f"/{path}")

train_targets = [i for i in train_df.disease_code]

datamodule = ImageClassificationData.from_files(
    train_files = train_files,
    train_targets = train_targets,
    predict_files = predict_files,
)

In [9]:
#predict_files[:10]

In [10]:
# build the task
model = 0
model = ImageClassifier(backbone="resnet101",num_classes=7)


In [11]:
# create the trainer and finetune the model
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | train_metrics | ModuleDict | 0     
1 | val_metrics   | ModuleDict | 0     
2 | backbone      | ResNet     | 42.5 M
3 | head          | Sequential | 14.3 K
---------------------------------------------
119 K     Trainable params
42.4 M    Non-trainable params
42.5 M    Total params
170.058   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for de

In [13]:
len(predict_files[])

4750

In [ ]:
model.eval()

In [17]:
datamodule.

In [ ]:
submission = pd.read_csv(PATH + '/sample_submission.csv')
submission.iloc[:,1] = preds
submission.to_csv(PATH + '/flash_test.csv', index=False)

In [ ]:
!flash image_classification --help